In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = mls
band = [3]
commitnumber = de5d43e
molecule = {'n2o': 3.2e-07}
tsfc = 294


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o n2o band3a
  atmpro = mls
  band = 3a
  commitnumber = a06b618
  conc = 3.2e-07
  dv = 0.001
  klin = 2.22e-20
  molecule = n2o
  ng_adju = [0, 0]
  ng_refs = [1, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (500, 250)]
  tsfc = 294
  vmax = 620
  vmin = 540
  w_diffuse = [(1.8,), (1.66, 1.8)]
  wgt = [(0.9,), (0.5, 0.95)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0  0.000526                  NaN       NaN
0.000312      NaN       NaN                  1.0  0.005308
0.000750      2.0  0.000688                  2.0  0.005436
0.001052      3.0  0.001041                  3.0  0.005727
0.001476      4.0  0.001417                  4.0  0.006031
0.002070      5.0  0.001795                  5.0  0.006337
0.002904      6.0  0.002174                  6.0  0.006645
0.004074      7.0  0.002554                  7.0  0.006955
0.005714      8.0  0.002935                  8.0  0.007267
0.008015      9.0  0.003315                  9.0  0.007582
0.011243     10.0  0.003695                 10.0  0.007898
0.015771     11.0  0.004053                 11.0  0.008214
0.022122     12.0  0.004395                 12.0  0.008532
0.031031     13.0  0.004707                 13.0  0.008851
0.043528     14.0  0.005017                 14.0  0.009192
0.061057     15.0  0.006143                 15.0  0.010221
0.085645     16.0  0.009525                 16.0  0.013099
0.120136     17.0  0.014702                 17.0  0.017461
0.168516     18.0  0.020390                 18.0  0.022299
0.236378     19.0  0.026503                 19.0  0.027544
0.331549     20.0  0.033050                 20.0  0.033200
0.465100     21.0  0.040020                 21.0  0.039282
0.652400     22.0  0.047425                 22.0  0.045751
0.915100     23.0  0.053434                 23.0  0.051112
1.283650     24.0  0.053157                 24.0  0.051427
1.800600     25.0  0.047109                 25.0  0.046979
2.525700     26.0  0.039992                 26.0  0.041349
3.542800     27.0  0.033261                 27.0  0.035592
4.969550     28.0  0.027280                 28.0  0.029989
6.970850     29.0  0.022233                 29.0  0.024696
9.778100     30.0  0.017949                 30.0  0.019680
13.715850    31.0  0.013980                 31.0  0.014690
19.239350    32.0  0.010161                 32.0  0.009827
26.987250    33.0  0.006887                 33.0  0.005591
37.855300    34.0  0.004119                 34.0  0.002107
53.100050    35.0  0.001196                 35.0 -0.001048
73.887500    36.0 -0.001897                 36.0 -0.003864
97.662500    37.0 -0.004242                 37.0 -0.005761
121.437500   38.0 -0.005814                 38.0 -0.006911
145.212500   39.0 -0.007326                 39.0 -0.007920
168.987500   40.0 -0.008400                 40.0 -0.008524
192.762500   41.0 -0.007962                 41.0 -0.007940
216.537500   42.0 -0.006223                 42.0 -0.006324
240.312500   43.0 -0.004348                 43.0 -0.004542
264.087500   44.0 -0.002584                 44.0 -0.002805
287.862500   45.0 -0.000915                 45.0 -0.001121
311.637500   46.0  0.000680                 46.0  0.000516
335.412500   47.0  0.002213                 47.0  0.002106
359.187500   48.0  0.003692                 48.0  0.003652
382.962500   49.0  0.005123                 49.0  0.005156
406.737500   50.0  0.006502                 50.0  0.006614
430.512500   51.0  0.007826                 51.0  0.008018
454.287500   52.0  0.009089                 52.0  0.009366
478.062500   53.0  0.010293                 53.0  0.010660
501.837500   54.0  0.011443                 54.0  0.011904
525.612500   55.0  0.012554                 55.0  0.013097
549.387500   56.0  0.013636                 56.0  0.014260
573.162500   57.0  0.014695                 57.0  0.015397
596.937500   58.0  0.015737                 58.0  0.016512
620.712500   59.0  0.016763                 59.0  0.017608
644.487500   60.0  0.017778                 60.0  0.018684
668.262500   61.0  0.018779                 61.0  0.019740
692.037500   62.0  0.019770                 62.0  0.020782
715.812500   63.0  0.020753                 63.0  0.021809
739.587500   64.0  0.021730                 64.0  0.022823
76

# Fluxes by Layer

CRD                          CLIRAD            \
                         flug      fldg      fnetg       flug      fldg   
pressure     level                                                        
0.000000e+00 1     -35.017092  0.000000 -35.017092        NaN       NaN   
1.000000e-08 1            NaN       NaN        NaN -35.064029  0.000002   
6.244000e-04 2     -35.017093  0.000001 -35.017091 -35.064030  0.000003   
8.759000e-04 3     -35.017093  0.000002 -35.017091 -35.064030  0.000003   
1.228600e-03 4     -35.017094  0.000003 -35.017091 -35.064031  0.000004   
1.723400e-03 5     -35.017095  0.000004 -35.017091 -35.064031  0.000005   
2.417400e-03 6     -35.017096  0.000005 -35.017091 -35.064032  0.000006   
3.390900e-03 7     -35.017098  0.000007 -35.017091 -35.064033  0.000008   
4.756500e-03 8     -35.017101  0.000010 -35.017091 -35.064034  0.000010   
6.672000e-03 9     -35.017105  0.000015 -35.017090 -35.064036  0.000014   
9.358900e-03 10    -35.017110  0.000021 -35.017089 -35.064039  0.000019   
1.312780e-02 11    -35.017117  0.000030 -35.017087 -35.064043  0.000026   
1.841450e-02 12    -35.017127  0.000042 -35.017085 -35.064048  0.000037   
2.583020e-02 13    -35.017140  0.000060 -35.017081 -35.064055  0.000051   
3.623230e-02 14    -35.017159  0.000084 -35.017075 -35.064065  0.000072   
5.082340e-02 15    -35.017186  0.000119 -35.017066 -35.064078  0.000101   
7.129060e-02 16    -35.017221  0.000170 -35.017051 -35.064095  0.000143   
1.000000e-01 17    -35.017265  0.000246 -35.017019 -35.064115  0.000207   
1.402710e-01 18    -35.017315  0.000366 -35.016949 -35.064133  0.000308   
1.967600e-01 19    -35.017365  0.000553 -35.016812 -35.064141  0.000466   
2.759970e-01 20    -35.017406  0.000842 -35.016564 -35.064130  0.000713   
3.871000e-01 21    -35.017417  0.001289 -35.016129 -35.064077  0.001097   
5.431000e-01 22    -35.017364  0.001974 -35.015389 -35.063948  0.001694   
7.617000e-01 23    -35.017180  0.003019 -35.014161 -35.063685  0.002616   
1.068500e+00 24    -35.016790  0.004570 -35.012219 -35.063219  0.004008   
1.498800e+00 25    -35.016200  0.006691 -35.009510 -35.062554  0.005965   
2.102400e+00 26    -35.015510  0.009369 -35.006141 -35.061772  0.008542   
2.949000e+00 27    -35.014791  0.012660 -35.002131 -35.060938  0.011856   
4.136600e+00 28    -35.014125  0.016674 -34.997452 -35.060145  0.016069   
5.802500e+00 29    -35.013646  0.021578 -34.992068 -35.059537  0.021380   
8.139200e+00 30    -35.013558  0.027644 -34.985914 -35.059346  0.028026   
1.141700e+01 31    -35.014208  0.035263 -34.978945 -35.059956  0.036278   
1.601470e+01 32    -35.016239  0.044908 -34.971331 -35.062053  0.046376   
2.246400e+01 33    -35.020734  0.057166 -34.963568 -35.066751  0.058583   
3.151050e+01 34    -35.029189  0.073002 -34.956187 -35.075627  0.073451   
4.420010e+01 35    -35.043769  0.093773 -34.949996 -35.090981  0.091972   
6.200000e+01 36    -35.068290  0.120815 -34.947475 -35.116636  0.115418   
8.577500e+01 37    -35.106919  0.154101 -34.952817 -35.156414  0.144313   
1.095500e+02 38    -35.150422  0.185658 -34.964764 -35.200558  0.172229   
1.333250e+02 39    -35.197626  0.216489 -34.981138 -35.247905  0.200109   
1.571000e+02 40    -35.248429  0.246660 -35.001769 -35.298151  0.228049   
1.808750e+02 41    -35.302299  0.276873 -35.025426 -35.350641  0.256531   
2.046500e+02 42    -35.357122  0.309272 -35.047850 -35.403617  0.287143   
2.284250e+02 43    -35.410858  0.345482 -35.065376 -35.455484  0.321198   
2.522000e+02 44    -35.462978  0.385357 -35.077621 -35.505839  0.358760   
2.759750e+02 45    -35.513391  0.428493 -35.084898 -35.554581  0.399601   
2.997500e+02 46    -35.562055  0.474581 -35.087475 -35.601650  0.443513   
3.235250e+02 47    -35.608943  0.523382 -35.085560 -35.647004  0.490322   
3.473000e+02 48    -35.654033  0.574705 -35.079329 -35.690598  0.539847   
3.710750e+02 49    -35.697313  0.628383 -35.068930 -35.732409  0.591944   
3.948500e+02 50    -35.738775  0.684274 -35.054502 

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')